# Using motleycrew with Autogen

In [2]:
import autogen
import os

llm_config = {
    "config_list": [{"model": "gpt-4-turbo", "api_key": os.environ["OPENAI_API_KEY"]}],
    "cache_seed": 2,
}

In [3]:
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="TERMINATE",
)
coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llm_config,
)
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Creative in software product ideas.",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, coder, pm], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [4]:
from langchain.tools import Tool

def retrieve_knowledge_by_topic(topic: str):
    chat_result = user_proxy.initiate_chat(
        manager,
        message=f"Find a latest paper about {topic} on arxiv "
                 "and find its potential applications in software.")

    for message in reversed(chat_result.chat_history):
        if message.get("content") and "TERMINATE" not in message["content"]:
            return message["content"]


knowledge_retrieval_tool = Tool.from_function(
    retrieve_knowledge_by_topic,
    name="Retrieve Knowledge by Topic",
    description="Search arxiv for the latest paper on a given topic "
                "and find its potential applications in software.",
)

In [5]:
from motleycrew import MotleyCrew
from motleycrew.agents.langchain import ReactMotleyAgent

crew = MotleyCrew()
writer = ReactMotleyAgent(tools=[knowledge_retrieval_tool])

In [6]:
from motleycrew.tasks import SimpleTaskRecipe

blog_post_task = SimpleTaskRecipe(
        name="Produce blog post on latest advancements related to GPT-4",
        description="Using the insights provided by searching research papers, develop an engaging blog "
                    "post that highlights the most significant advancements on GPT-4 ant their applications.\n"
                    "Your post should be informative yet accessible, catering to a tech-savvy audience.\n"
                    "Make it sound cool, avoid complex words so it doesn't sound like AI. "
                    "Create a blog post of at least 4 paragraphs.",
        agent=writer,
    )
crew.register_task_recipes([blog_post_task])

In [7]:
crew.run()

User_proxy (to chat_manager):

Find a latest paper about GPT-4 advancements and applications on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------
Coder (to chat_manager):

To locate the most recent paper about GPT-4 advancements and applications on arXiv, we can use Python to programmatically access the arXiv API and search for papers related to "GPT-4". 

Here's a Python script you can run to fetch the title and abstract of the most recent paper from arXiv on GPT-4 advancements and applications:

```python
# filename: fetch_latest_gpt4_paper.py
import requests
from urllib.parse import quote

def fetch_recent_gpt4_papers():
    base_url = "http://export.arxiv.org/api/query?search_query="
    query = "all:GPT-4+AND+ti:advancements+AND+ti:applications"

    # Encode the query for URL
    query = quote(query)
    url = f"{base_url}{query}&sortBy=submittedDate&sortOrder=descending&max_results=1"
    resp

[Task(status=done)]

In [8]:
from IPython.display import display, Markdown
display(Markdown(blog_post_task.output))

**Exploring the Frontier of AI with GPT-4: A Leap into the Future**

Welcome to the cutting-edge world of GPT-4, the latest iteration in the series of groundbreaking language models developed by OpenAI. As we dive into the capabilities and applications of this advanced tool, it's clear that GPT-4 is not just an incremental update but a transformative leap that is reshaping how we interact with technology.

GPT-4 has taken the tech world by storm, primarily due to its enhanced understanding and generation of human-like text. This capability makes it an invaluable asset across various sectors. In customer service, for instance, GPT-4-powered chatbots are now more adept than ever at handling complex queries with a level of nuance and context awareness previously unattainable. This means smoother interactions for customers and less strain on human resources.

The impact of GPT-4 extends into the realm of software development as well. Developers can rejoice as they integrate GPT-4 into their IDEs, where it assists not only in code completion but also in identifying potential bugs and optimizing code performance on the fly. This integration marks a significant step towards more efficient and less error-prone coding practices.

For content creators, GPT-4 is nothing short of a revolution. Whether it's drafting blog posts, scripting videos, or creating engaging marketing content, GPT-4 helps streamline the creative process by generating initial drafts and suggesting improvements. This allows creators to focus more on refining their ideas and less on the mundane aspects of content generation.

In the educational sector, GPT-4's ability to tailor content and assessments to individual learning styles and paces is transforming e-learning platforms. Students receive a more personalized learning experience, which can adapt in real-time to their educational needs, enhancing both engagement and retention rates.

As we continue to explore and integrate GPT-4's capabilities, the potential applications seem almost limitless. From enhancing interactive entertainment and gaming to revolutionizing simulations and training for professionals, GPT-4 is setting the stage for a more interactive and responsive technological future. Stay tuned as we continue to uncover the full potential of this extraordinary AI advancement.